In [1]:
from kerastuner import HyperModel
import numpy as np
from kerastuner.tuners import RandomSearch
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
class HyperRAPID(HyperModel):

    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        
        model = keras.Sequential()
        model.add(layers.Masking(mask_value=0.))
        for i in range(hp.Int('num_layers', 1, 3)):
            model.add(layers.LSTM(hp.Choice('units', [25,50,100]), return_sequences=True, dropout=hp.Choice('dropout', [0.0,0.1,0.2,0.3])))
        model.add(layers.TimeDistributed(layers.Dense(self.num_classes, activation='softmax')))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model

In [3]:
files = np.load('/home/miranda/ztf-rapid/data/processed/test_none/test_none.npz')

In [4]:
[key for key in files.keys()]

['X_train', 'X_test', 'y_train', 'y_test', 'objids_test', 'class_names']

In [5]:
y_train = files['y_train'][:,:,1:]
y_test = files['y_test'][:,:,1:]

In [6]:
num_classes = len(files['class_names'])
files['class_names']

array(['AGN', 'CV', 'SN II', 'SN Ia'], dtype='<U5')

In [7]:
hypermodel = HyperRAPID(num_classes)

In [8]:
tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=1,
    directory='test',
    project_name='test')

INFO:tensorflow:Reloading Oracle from existing project test/test/oracle.json


In [9]:
tuner.search(files['X_train'], y_train, epochs=1, validation_data=(files['X_test'], y_test))

Train on 2420 samples, validate on 1616 samples
2420/2420 [==============================] - ETA: 18:23 - loss: 0.5111 - accuracy: 0.117 - ETA: 9:42 - loss: 0.4525 - accuracy: 0.405 - ETA: 6:47 - loss: 0.4340 - accuracy: 0.45 - ETA: 5:20 - loss: 0.4260 - accuracy: 0.47 - ETA: 4:27 - loss: 0.3906 - accuracy: 0.55 - ETA: 3:51 - loss: 0.3787 - accuracy: 0.56 - ETA: 3:26 - loss: 0.3909 - accuracy: 0.54 - ETA: 3:06 - loss: 0.4108 - accuracy: 0.52 - ETA: 2:51 - loss: 0.4061 - accuracy: 0.55 - ETA: 2:38 - loss: 0.4190 - accuracy: 0.52 - ETA: 2:27 - loss: 0.4082 - accuracy: 0.55 - ETA: 2:19 - loss: 0.3981 - accuracy: 0.56 - ETA: 2:11 - loss: 0.3978 - accuracy: 0.56 - ETA: 2:04 - loss: 0.3938 - accuracy: 0.57 - ETA: 1:58 - loss: 0.3899 - accuracy: 0.57 - ETA: 1:53 - loss: 0.3946 - accuracy: 0.56 - ETA: 1:48 - loss: 0.3931 - accuracy: 0.56 - ETA: 1:44 - loss: 0.3931 - accuracy: 0.56 - ETA: 1:39 - loss: 0.3902 - accuracy: 0.56 - ETA: 1:36 - loss: 0.3866 - accuracy: 0.56 - ETA: 1:32 - loss: 0.3867

INFO:tensorflow:Oracle triggered exit


In [10]:
tuner.results_summary()

In [13]:
best_hp = tuner.get_best_hyperparameters()[0]

In [14]:
best_hp.values

{'num_layers': 3, 'units': 100, 'dropout': 0.2}